In [1]:
import sys
import os
%load_ext autoreload
%autoreload 2

sys.path.append('../')
os.chdir('../')

In [2]:
import re

string = "res=[0, 1, 2, 5]"
pattern = r"res=\[(.*?)\]"
result = re.findall(pattern, string)[0]
lst = [int(num) for num in result.split(", ")]
print(lst)


[0, 1, 2, 5]


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import re


global_path = './outputs'
all_result_paths = glob.glob(global_path + '/*_exp/**/result.csv', recursive=True)
result_summary = {}
n_layers = 6
for result_path in all_result_paths:
    result = pd.read_csv(result_path)
    top_test_result = result['test_acc'].max()
    pe_type = 'NoPE' if 'nope' in result_path else 'Original'
    pattern = r"res=\[(.*?)\]"
    residual_list = re.findall(pattern, result_path)[0]
    residual_list = [int(num) for num in residual_list.split(", ")]
    missing_rc = [i for i in range(n_layers) if i not in residual_list]
    missing_rc_str = '{' + ','.join([str(i) for i in missing_rc])+ '}'


    if pe_type not in result_summary:
        result_summary[pe_type] = {}
    if missing_rc_str not in result_summary[pe_type]:
        result_summary[pe_type][missing_rc_str] = []
    result_summary[pe_type][missing_rc_str].append(top_test_result)
    if pe_type == 'Original' and missing_rc_str == '{5}':
        print(result_path, pe_type, missing_rc_str, top_test_result)


./outputs/oddc_residual_exp/oddc_sd240_T2403201643_res=[0, 1, 2, 3, 4]/oddc_sd240_T2403201643_res=[0, 1, 2, 3, 4]/result.csv Original {5} 100.0
./outputs/oddc_residual_exp/oddc_sd241_T2403202103_res=[0, 1, 2, 3, 4]/oddc_sd241_T2403202103_res=[0, 1, 2, 3, 4]/result.csv Original {5} 100.0
./outputs/oddc_residual_exp/oddc_sd242_T2403202348_res=[0, 1, 2, 3, 4]/oddc_sd242_T2403202348_res=[0, 1, 2, 3, 4]/result.csv Original {5} 99.99
./outputs/oddc_residual_exp/oddc_sd243_T2403210232_res=[0, 1, 2, 3, 4]/oddc_sd243_T2403210232_res=[0, 1, 2, 3, 4]/result.csv Original {5} 100.0
./outputs/oddc_residual_exp/oddc_sd244_T2403210515_res=[0, 1, 2, 3, 4]/oddc_sd244_T2403210515_res=[0, 1, 2, 3, 4]/result.csv Original {5} 100.0


In [ ]:
result_summary

In [5]:
from ast import literal_eval
mean_results = {}
for pe_type in result_summary:
    cur_type_dict = result_summary[pe_type].copy()
    for missing_rc in cur_type_dict:
        cur_type_dict[missing_rc] = len(cur_type_dict[missing_rc])
    mean_results[pe_type] = cur_type_dict
df = pd.DataFrame(mean_results)
missing0 = ['{}']
missing1 = [str({i}) for i in range(n_layers)]
missing2 = ['{' + ','.join([str(i), str(i+1)])+ '}'for i in range(n_layers-1)]
missing3 = ['{' + ','.join([str(i), str(i+1), str(i+2)])+ '}'for i in range(n_layers-2)]
from IPython.display import display
display(df.loc[missing0,:].T)
display(df.loc[missing1,:].T)
display(df.loc[missing2,:].T)
display(df.loc[missing3,:].T)

,{}
NoPE,4
Original,4


,{0},{1},{2},{3},{4},{5}
NoPE,5,5,5,4,4,4
Original,5,5,5,5,5,5


,"{0,1}","{1,2}","{2,3}","{3,4}","{4,5}"
NoPE,10,10,10,10,10
Original,10,10,10,10,10


,"{0,1,2}","{1,2,3}","{2,3,4}","{3,4,5}"
NoPE,10,10,10,10
Original,10,10,10,10


In [7]:
from ast import literal_eval
from collections import OrderedDict
final_results = OrderedDict()
for pe_type in {'Original', 'NoPE'}:
    cur_type_dict = result_summary[pe_type].copy()
    final_dict = {
        'min': cur_type_dict.copy(),
        'max': cur_type_dict.copy(),
        'avg.': cur_type_dict.copy(),
    }
    for missing_rc in cur_type_dict:
        final_dict['avg.'][missing_rc] = np.mean(cur_type_dict[missing_rc])
        final_dict['min'][missing_rc] = np.min(cur_type_dict[missing_rc])
        final_dict['max'][missing_rc] = np.max(cur_type_dict[missing_rc])

    for k in final_dict:
        final_results[pe_type+' ('+k+')'] = final_dict[k]
df = pd.DataFrame(final_results)
missing1 = [str({i}) for i in range(n_layers)]

missing2 = ['{' + ','.join([str(i), str(i+1)])+ '}'for i in range(n_layers-1)]
missing3 = ['{' + ','.join([str(i), str(i+1), str(i+2)])+ '}'for i in range(n_layers-2)]
from IPython.display import display
r1 = df.loc[missing1,:].T
r2 = df.loc[missing2,:].T
r3 = df.loc[missing3,:].T
r0 = df.loc[['{}'],:].T
display(r0)
display(r1)
display(r2)
display(r3)

# write all 3 to the same csv file
all_r = pd.concat([r0, r1, r2, r3], axis=1)
all_r.to_csv('all_r_mean.csv')


,{}
Original (min),100.00
Original (max),100.00
Original (avg.),100.00
NoPE (min),99.07
NoPE (max),99.66
NoPE (avg.),99.40


,{0},{1},{2},{3},{4},{5}
Original (min),99.980,99.990,100.000,99.9800,100.00,99.9900
Original (max),100.000,100.000,100.000,100.0000,100.00,100.0000
Original (avg.),99.994,99.998,100.000,99.9940,100.00,99.9980
NoPE (min),99.890,99.550,99.670,98.5900,99.37,99.7100
NoPE (max),99.970,99.820,99.990,99.5600,99.77,99.9400
NoPE (avg.),99.930,99.724,99.882,99.0525,99.49,99.8425


,"{0,1}","{1,2}","{2,3}","{3,4}","{4,5}"
Original (min),99.980,2.930,0.440,0.440,0.430
Original (max),100.000,100.000,24.840,24.610,49.150
Original (avg.),99.990,67.391,8.239,7.741,11.954
NoPE (min),99.420,3.350,0.980,0.460,0.450
NoPE (max),99.960,99.460,96.570,70.210,99.190
NoPE (avg.),99.793,64.624,26.881,14.970,24.907


,"{0,1,2}","{1,2,3}","{2,3,4}","{3,4,5}"
Original (min),0.470,0.440,0.440,0.430
Original (max),25.030,0.540,0.560,0.550
Original (avg.),10.446,0.480,0.499,0.480
NoPE (min),0.470,0.430,0.420,0.420
NoPE (max),24.950,24.040,0.570,0.610
NoPE (avg.),11.605,3.641,0.492,0.479
